In [1]:
import numpy as np
from aeon.distances import mpdist

In [19]:
y = np.array([[42, 23, 21, 55, 1, 19, 33, 34, 29, 19]])
print(y.shape)
if y.ndim == 2 and y.shape[0] > 1:
    raise ValueError("y must be a 1D array or shape (1,n)")
y = y.squeeze()
print(y.shape)

(1, 10)
(10,)


In [17]:
y

array([42, 23, 21, 55,  1, 19, 33, 34, 29, 19])

In [10]:
y = np.array([[11, 12, 13, 14],[7, 8, 9, 20],[1, 3, 4, 5]])
print(y.shape)
if y.ndim == 2 and y.shape[0] > 1:
    raise ValueError("y must be a 1D array or shape (1,n)")

(3, 4)


ValueError: y must be a 1D array or shape (1,n)

In [14]:
y = np.array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
print(y.shape)
if y.ndim == 2 and y.shape[0] > 1:
    raise ValueError("y must be a 1D array or shape (1,n)")
y = y.squeeze()
print(y.shape)

(10,)
(10,)


In [15]:
y

array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [20]:
y = np.array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [22]:
if y.ndim == 1:
    _y = y.reshape((1, y.shape[0]))
_y.shape

(1, 10)

In [24]:
x = np.array([1,2,3,5,9])
y = np.array([3,7,13,19])
m = 2

In [25]:
len(x)

5

In [26]:
n_x_subs = len(x) - m + 1
n_y_subs = len(y) - m + 1

In [27]:
x_mean = [np.mean(x[i : i + m]) for i in range(0, n_x_subs)]
x_mean

[1.5, 2.5, 4.0, 7.0]

In [29]:
# Compute the mean and standard deviation
x_mean = [np.mean(x[i : i + m]) for i in range(0, n_x_subs)]
x_std = [np.std(x[i : i + m]) for i in range(0, n_x_subs)]

y_mean = [np.mean(y[i : i + m]) for i in range(0, n_y_subs)]
y_std = [np.std(y[i : i + m]) for i in range(0, n_y_subs)]

In [31]:
print(x_mean)
print(x_std)
print(y_mean)
print(y_std)

[1.5, 2.5, 4.0, 7.0]
[0.5, 0.5, 1.0, 2.0]
[5.0, 10.0, 16.0]
[2.0, 3.0, 3.0]
